In [1]:
!nvidia-smi

Wed Feb 25 05:45:00 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
 !git clone https://github.com/Judinus10/lesion-aware-dr.git 
%cd lesion-aware-dr

Cloning into 'lesion-aware-dr'...
remote: Enumerating objects: 201, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 201 (delta 94), reused 178 (delta 71), pack-reused 0 (from 0)
Receiving objects: 100% (201/201), 103.95 KiB | 4.00 MiB/s, done.
Resolving deltas: 100% (94/94), done.
/kaggle/working/lesion-aware-dr


In [3]:
DATA_ROOT="/kaggle/input/datasets/mohlamin/resized-eyepacs-diabetic-retinopathy-dataset"

!echo $DATA_ROOT
!ls -lah $DATA_ROOT
!ls -lah $DATA_ROOT/Images | head

/kaggle/input/datasets/mohlamin/resized-eyepacs-diabetic-retinopathy-dataset
total 3.2M
drwxr-xr-x 3 nobody nogroup    0 Feb 16 16:26 .
drwxr-xr-x 3 root   root    4.0K Feb 25 05:44 ..
-rw-r--r-- 1 nobody nogroup 1.6M Feb 16 16:26 all_labels.csv
drwxr-xr-x 2 nobody nogroup    0 Feb 16 16:26 Images
-rw-r--r-- 1 nobody nogroup 1.2M Feb 16 16:26 original_test_labels.csv
-rw-r--r-- 1 nobody nogroup 455K Feb 16 16:26 original_train_labels.csv
total 23G
drwxr-xr-x 2 nobody nogroup    0 Feb 16 16:26 .
drwxr-xr-x 3 nobody nogroup    0 Feb 16 16:26 ..
-rw-r--r-- 1 nobody nogroup 218K Feb 16 16:19 10000_left.png
-rw-r--r-- 1 nobody nogroup 214K Feb 16 16:19 10000_right.png
-rw-r--r-- 1 nobody nogroup 216K Feb 16 16:19 10001_left.png
-rw-r--r-- 1 nobody nogroup 211K Feb 16 16:19 10001_right.png
-rw-r--r-- 1 nobody nogroup 186K Feb 16 16:19 10002_left.png
-rw-r--r-- 1 nobody nogroup 297K Feb 16 16:19 10002_right.png
-rw-r--r-- 1 nobody nogroup 210K Feb 16 16:19 10003_left.png
ls: write error: Brok

In [4]:
from pathlib import Path

file_path = Path("/kaggle/working/lesion-aware-dr/scripts/make_split.py")

print(file_path.read_text()[:500])

import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path

# --------------------
# Paths
# --------------------
DATA_DIR = Path(r"E:\DR_related\eyepacs")
LABELS_CSV = DATA_DIR / "all_labels.csv"
IMAGES_DIR = DATA_DIR / "Images"

OUT_TRAIN = DATA_DIR / "train.csv"
OUT_VAL = DATA_DIR / "val.csv"

# --------------------
# Load labels
# --------------------
df = pd.read_csv(LABELS_CSV)
print("Original columns:", list(df.columns))

# --------------------
# Nor


In [5]:
from pathlib import Path

file_path = Path("/kaggle/working/lesion-aware-dr/scripts/make_split.py")

text = file_path.read_text()

text = text.replace(
    r"E:\DR_related\eyepacs",
    "/kaggle/input/datasets/mohlamin/resized-eyepacs-diabetic-retinopathy-dataset"
)

file_path.write_text(text)

print("✅ Path updated")

✅ Path updated


In [6]:
text = file_path.read_text()

text = text.replace(
    'DATA_DIR / "train.csv"',
    'Path("/kaggle/working/train.csv")'
)

text = text.replace(
    'DATA_DIR / "val.csv"',
    'Path("/kaggle/working/val.csv")'
)

file_path.write_text(text)

print("✅ Output paths updated")

✅ Output paths updated


In [7]:
%cd /kaggle/working/lesion-aware-dr
!python scripts/make_split.py

/kaggle/working/lesion-aware-dr
Original columns: ['image', 'level', 'Usage']
✅ All image files found.
Saved: /kaggle/working/train.csv
Saved: /kaggle/working/val.csv
Train size: 70960
Val size: 17740

Train label distribution:
label
0    52273
1     4964
2    10522
3     1670
4     1531
Name: count, dtype: int64

Val label distribution:
label
0    13069
1     1241
2     2630
3      417
4      383
Name: count, dtype: int64


In [8]:
!ls /kaggle/working

lesion-aware-dr  __notebook__.ipynb  train.csv	val.csv


In [9]:
from pathlib import Path
import re

cfg_path = Path("/kaggle/working/lesion-aware-dr/configs/base.yaml")
text = cfg_path.read_text(encoding="utf-8")

# ---------- 1) Force paths block ----------
new_paths_block = """paths:
  data_dir: "/kaggle/working"
  raw_dir: "/kaggle/working"
  processed_dir: "/kaggle/working"
  outputs_dir: "/kaggle/working/outputs"
  checkpoints_dir: "/kaggle/working/outputs/checkpoints"
"""

# Replace existing paths block if present, else prepend it
if re.search(r"(?m)^paths:\n(?:^[ \t].*\n)+", text):
    text = re.sub(r"(?m)^paths:\n(?:^[ \t].*\n)+", new_paths_block + "\n", text)
else:
    text = new_paths_block + "\n" + text

# ---------- 2) Force loss + sampler settings ----------
desired = {
    "loss.name": "focal",
    "loss.gamma": "1.5",
    "loss.use_class_weights": "true",
    "data.use_weighted_sampler": "false",
}

def set_key(cfg_text: str, key: str, value: str) -> str:
    section, field = key.split(".")

    # Replace field inside existing section block
    pattern = rf"(?ms)^({section}:\n(?:^[ \t].*\n)*)^[ \t]*{field}:\s*.*$"
    repl = rf"\1  {field}: {value}"
    if re.search(pattern, cfg_text):
        return re.sub(pattern, repl, cfg_text)

    # If section exists but field missing: insert after section header
    sec_header = rf"(?m)^{section}:\s*$"
    if re.search(sec_header, cfg_text):
        return re.sub(sec_header, f"{section}:\n  {field}: {value}", cfg_text, count=1)

    # If section missing entirely: append section at end
    return cfg_text.rstrip() + f"\n\n{section}:\n  {field}: {value}\n"

for k, v in desired.items():
    text = set_key(text, k, v)

cfg_path.write_text(text, encoding="utf-8")
print("✅ Patched paths + loss + sampler in base.yaml")

# ---------- 3) Quick verification print ----------
print("\n--- paths block ---")
m = re.search(r"(?ms)^paths:\n(?:^[ \t].*\n)+", text)
print(m.group(0).strip() if m else "paths block not found")

print("\n--- data block (sampler lines) ---")
m = re.search(r"(?ms)^data:\n(?:^[ \t].*\n)+", text)
if m:
    # print only key lines
    data_lines = [ln for ln in m.group(0).splitlines() if "use_weighted_sampler" in ln or "sampler_mode" in ln]
    print("\n".join(data_lines) if data_lines else "(no sampler lines found)")
else:
    print("data block not found")

print("\n--- loss block ---")
m = re.search(r"(?ms)^loss:\n(?:^[ \t].*\n)+", text)
print(m.group(0).strip() if m else "loss block not found")

✅ Patched paths + loss + sampler in base.yaml

--- paths block ---
paths:
  data_dir: "/kaggle/working"
  raw_dir: "/kaggle/working"
  processed_dir: "/kaggle/working"
  outputs_dir: "/kaggle/working/outputs"
  checkpoints_dir: "/kaggle/working/outputs/checkpoints"


model:
  backbone: "efficientnet_b0"
  num_classes: 5
  pretrained: true

data:
  use_dummy: false
  image_size: 224

  train_csv: "/content/drive/MyDrive/Lesion_aware_DR/eyepacs/train.csv"
  val_csv:   "/content/drive/MyDrive/Lesion_aware_DR/eyepacs/val_clean.csv"
  image_dir: "/content/eyepacs_images/Images"
  image_col: "image_id"
  label_col: "label"

  # Use ONE balancing method at a time.
  # Here: sampler OFF, loss weights ON.

--- data block (sampler lines) ---
(no sampler lines found)

--- loss block ---
loss block not found


In [10]:
from pathlib import Path
import re

cfg_path = Path("/kaggle/working/lesion-aware-dr/configs/base.yaml")
text = cfg_path.read_text(encoding="utf-8")

new_paths_block = """paths:
  data_dir: "/kaggle/working"
  raw_dir: "/kaggle/working"
  processed_dir: "/kaggle/working"
  outputs_dir: "/kaggle/working/outputs"
  checkpoints_dir: "/kaggle/working/outputs/checkpoints"
"""

# Replace existing paths: block if present, else prepend it.
if re.search(r"(?m)^paths:\n(?:^[ \t].*\n)+", text):
    text = re.sub(r"(?m)^paths:\n(?:^[ \t].*\n)+", new_paths_block + "\n", text)
else:
    text = new_paths_block + "\n" + text

cfg_path.write_text(text, encoding="utf-8")
print("✅ Patched paths block in base.yaml")

✅ Patched paths block in base.yaml


In [11]:
!sed -n '1,120p' /kaggle/working/lesion-aware-dr/configs/base.yaml

project_name: "lesion_aware_dr"
run_name: "efficientnet_cb_focal"

paths:
  data_dir: "/kaggle/working"
  raw_dir: "/kaggle/working"
  processed_dir: "/kaggle/working"
  outputs_dir: "/kaggle/working/outputs"
  checkpoints_dir: "/kaggle/working/outputs/checkpoints"



model:
  backbone: "efficientnet_b0"
  num_classes: 5
  pretrained: true

data:
  use_dummy: false
  image_size: 224

  train_csv: "/content/drive/MyDrive/Lesion_aware_DR/eyepacs/train.csv"
  val_csv:   "/content/drive/MyDrive/Lesion_aware_DR/eyepacs/val_clean.csv"
  image_dir: "/content/eyepacs_images/Images"
  image_col: "image_id"
  label_col: "label"

  # Use ONE balancing method at a time.
  # Here: sampler OFF, loss weights ON.
  use_weighted_sampler: false

In [12]:
from pathlib import Path

req_path = Path("/kaggle/working/lesion-aware-dr/requirements_local.txt")

# try reading with utf-16 fallback
try:
    text = req_path.read_text(encoding="utf-8")
except UnicodeDecodeError:
    text = req_path.read_text(encoding="utf-16")

lines = text.splitlines()

remove_keys = [
    "torch==",
    "torchvision==",
    "torchaudio==",
    "opencv-python==",
]

cleaned = [line for line in lines if not any(k in line for k in remove_keys)]

needed = ["grad-cam", "torchmetrics", "einops"]

for pkg in needed:
    if not any(pkg in line for line in cleaned):
        cleaned.append(pkg)

req_path.write_text("\n".join(cleaned), encoding="utf-8")

print("✅ requirements fixed & converted to UTF-8")

✅ requirements fixed & converted to UTF-8


In [13]:
from pathlib import Path

repo = Path("/kaggle/working/lesion-aware-dr")

data_dir = repo / "src" / "data"
data_dir.mkdir(parents=True, exist_ok=True)

# make it a package
(data_dir / "__init__.py").write_text("", encoding="utf-8")

code = r'''from __future__ import annotations

from pathlib import Path
from typing import Dict, Any
import random

import numpy as np
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch import ToTensorV2


# ---------- Dummy dataset for quick testing ----------

class DummyDRDataset(Dataset):
    """
    Returns random images & labels.
    Use only when cfg.data.use_dummy = true.
    """
    def __init__(self, num_samples: int, num_classes: int, image_size: int):
        self.num_samples = int(num_samples)
        self.num_classes = int(num_classes)
        self.image_size = int(image_size)

    def __len__(self) -> int:
        return self.num_samples

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        image = torch.rand(3, self.image_size, self.image_size)
        label = torch.randint(0, self.num_classes, (1,)).item()
        return {"image": image, "label": torch.tensor(label, dtype=torch.long)}


# ---------- Real dataset for APTOS / EyePACS / etc. ----------

class DRDataset(Dataset):
    """
    Generic DR dataset reading from CSV:
      - image filename column (e.g., '16028_left.png')
      - integer label column (0..num_classes-1)

    Robust to corrupted images: retries a few times and then errors clearly.
    """

    def __init__(
        self,
        csv_path: str,
        images_dir: str,
        image_col: str,
        label_col: str,
        image_size: int = 224,
        augment: bool = False,
        max_decode_retries: int = 20,
        log_bad_every: int = 50,
    ):
        self.df = pd.read_csv(csv_path).reset_index(drop=True)
        self.images_dir = Path(images_dir)
        self.image_col = str(image_col)
        self.label_col = str(label_col)
        self.image_size = int(image_size)

        self.max_decode_retries = int(max_decode_retries)
        self.log_bad_every = int(log_bad_every)
        self.bad_count = 0

        # ✅ EfficientNet pretrained expects ImageNet normalization
        imagenet_mean = (0.485, 0.456, 0.406)
        imagenet_std = (0.229, 0.224, 0.225)

        if augment:
            self.transform = A.Compose(
                [
                    A.Resize(image_size, image_size),
                    A.HorizontalFlip(p=0.5),
                    A.RandomBrightnessContrast(p=0.4),
                    # Replace ShiftScaleRotate with Affine (newer albumentations recommendation)
                    A.Affine(
                        scale=(0.95, 1.05),
                        translate_percent=(0.0, 0.05),
                        rotate=(-15, 15),
                        p=0.5,
                        mode=cv2.BORDER_REFLECT_101,
                    ),
                    A.Normalize(mean=imagenet_mean, std=imagenet_std),
                    ToTensorV2(),
                ]
            )
        else:
            self.transform = A.Compose(
                [
                    A.Resize(image_size, image_size),
                    A.Normalize(mean=imagenet_mean, std=imagenet_std),
                    ToTensorV2(),
                ]
            )

    def __len__(self) -> int:
        return len(self.df)

    def _read_image(self, img_path: Path):
        img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
        if img is None:
            return None
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        for _ in range(self.max_decode_retries):
            row = self.df.iloc[idx]
            fname = str(row[self.image_col])
            img_path = self.images_dir / fname

            image = self._read_image(img_path)

            if image is not None:
                label = int(row[self.label_col])
                augmented = self.transform(image=image)
                return {
                    "image": augmented["image"],
                    "label": torch.tensor(label, dtype=torch.long),
                }

            # Bad image encountered
            self.bad_count += 1
            if self.log_bad_every > 0 and self.bad_count % self.log_bad_every == 0:
                print(f"[WARN] Skipped {self.bad_count} corrupted images so far. Latest: {img_path}")

            idx = random.randint(0, len(self.df) - 1)

        raise RuntimeError(
            f"Too many unreadable images encountered (>{self.max_decode_retries} retries). "
            f"Check dataset integrity and image_dir. Last attempted: {img_path}"
        )


# ---------- DataModule wrapper ----------

class DRDataModule:
    def __init__(self, cfg):
        self.cfg = cfg
        self.batch_size = int(cfg.training.batch_size)
        self.num_workers = int(cfg.training.num_workers)
        self.num_classes = int(cfg.model.num_classes)
        self.image_size = int(cfg.data.image_size)

        self.use_dummy = bool(cfg.data.get("use_dummy", False))

        # Optional sampler (only if you intentionally enable it in YAML)
        self.use_weighted_sampler = bool(cfg.data.get("use_weighted_sampler", False))
        self.sampler_mode = str(cfg.data.get("sampler_mode", "inverse")).lower()

    def _pin_memory(self) -> bool:
        return torch.cuda.is_available()

    def _build_weighted_sampler_from_csv(self, csv_path: str, label_col: str) -> WeightedRandomSampler:
        df = pd.read_csv(csv_path)
        labels = df[label_col].astype(int).values

        class_counts = np.bincount(labels, minlength=self.num_classes).astype(np.float32)
        class_counts = np.maximum(class_counts, 1.0)

        # inverse-frequency sampling
        class_weights = 1.0 / class_counts
        sample_weights = class_weights[labels]
        sample_weights = torch.tensor(sample_weights, dtype=torch.double)

        return WeightedRandomSampler(
            weights=sample_weights,
            num_samples=len(sample_weights),
            replacement=True,
        )

    # ---- Dataloaders ----
    def train_dataloader(self) -> DataLoader:
        if self.use_dummy:
            ds = DummyDRDataset(
                num_samples=256,
                num_classes=self.num_classes,
                image_size=self.image_size,
            )
            return DataLoader(
                ds,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=self.num_workers,
                pin_memory=self._pin_memory(),
            )

        ds = DRDataset(
            csv_path=self.cfg.data.train_csv,
            images_dir=self.cfg.data.image_dir,
            image_col=self.cfg.data.image_col,
            label_col=self.cfg.data.label_col,
            image_size=self.image_size,
            augment=True,
            max_decode_retries=30,
            log_bad_every=50,
        )

        if self.use_weighted_sampler:
            sampler = self._build_weighted_sampler_from_csv(
                csv_path=self.cfg.data.train_csv,
                label_col=self.cfg.data.label_col,
            )
            return DataLoader(
                ds,
                batch_size=self.batch_size,
                shuffle=False,  # sampler + shuffle cannot both be True
                sampler=sampler,
                num_workers=self.num_workers,
                pin_memory=self._pin_memory(),
            )

        return DataLoader(
            ds,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=self._pin_memory(),
        )

    def val_dataloader(self) -> DataLoader:
        if self.use_dummy:
            ds = DummyDRDataset(
                num_samples=64,
                num_classes=self.num_classes,
                image_size=self.image_size,
            )
        else:
            ds = DRDataset(
                csv_path=self.cfg.data.val_csv,
                images_dir=self.cfg.data.image_dir,
                image_col=self.cfg.data.image_col,
                label_col=self.cfg.data.label_col,
                image_size=self.image_size,
                augment=False,
                max_decode_retries=30,
                log_bad_every=50,
            )

        return DataLoader(
            ds,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=self._pin_memory(),
        )
'''

(data_dir / "dr_datamodule.py").write_text(code, encoding="utf-8")

print("✅ Created:", data_dir / "dr_datamodule.py")

✅ Created: /kaggle/working/lesion-aware-dr/src/data/dr_datamodule.py


In [14]:
from pathlib import Path
p = Path("/kaggle/working/lesion-aware-dr/requirements_local.txt")
print(p.read_text()[:200])

albucore==0.0.24
albumentations==2.0.8
altair==5.5.0
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==4.12.0
attrs==25.4.0
blinker==1.9.0
cachetools==6.2.2
certifi==2025.11.12
charset-norma


In [15]:
!pip install -q -r /kaggle/working/lesion-aware-dr/requirements_local.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 16.5 M

In [16]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

CUDA available: True
GPU: Tesla T4


In [17]:
from pathlib import Path

cfg_path = Path("/kaggle/working/lesion-aware-dr/configs/base.yaml")

clean_yaml = """project_name: "lesion_aware_dr"
run_name: "efficientnet_focal_g15_cw"

paths:
  data_dir: "/kaggle/working"
  raw_dir: "/kaggle/working"
  processed_dir: "/kaggle/working"
  outputs_dir: "/kaggle/working/outputs"
  checkpoints_dir: "/kaggle/working/outputs/checkpoints"

model:
  backbone: "efficientnet_b0"
  num_classes: 5
  pretrained: true

data:
  use_dummy: false
  image_size: 224

  train_csv: "/kaggle/working/train.csv"
  val_csv:   "/kaggle/working/val.csv"
  image_dir: "/kaggle/input/datasets/mohlamin/resized-eyepacs-diabetic-retinopathy-dataset/Images"
  image_col: "image_id"
  label_col: "label"

  use_weighted_sampler: false
  sampler_mode: "inverse"

training:
  seed: 42
  epochs: 12
  batch_size: 16
  num_workers: 2
  lr: 1e-4
  weight_decay: 1e-5
  optimizer: "adamw"
  scheduler: "cosine"

logging:
  use_wandb: false
  wandb_project: "lesion_aware_dr"
  wandb_entity: null

loss:
  name: "focal"
  beta: 0.999
  gamma: 1.5
  alpha_mode: "effective"
  reduction: "mean"
  use_class_weights: true
"""

cfg_path.write_text(clean_yaml, encoding="utf-8")
print("✅ Rewrote base.yaml cleanly:", cfg_path)

✅ Rewrote base.yaml cleanly: /kaggle/working/lesion-aware-dr/configs/base.yaml


In [18]:
%cd /kaggle/working/lesion-aware-dr
!python -m src.train --cfg_path configs/base.yaml

/kaggle/working/lesion-aware-dr
[2026-02-25 05:47:58] [INFO] Starting training script
[2026-02-25 05:47:58] [INFO] Using device: cuda
[2026-02-25 05:47:58] [INFO] [RUN DIR] New run folder: /kaggle/working/outputs/2026-02-25
[2026-02-25 05:47:58] [INFO] [RUN DIR] Checkpoints folder: /kaggle/working/outputs/2026-02-25/checkpoints
[2026-02-25 05:47:58] [INFO] Config:
project_name: lesion_aware_dr
run_name: efficientnet_focal_g15_cw
paths:
  data_dir: /kaggle/working
  raw_dir: /kaggle/working
  processed_dir: /kaggle/working
  outputs_dir: /kaggle/working/outputs/2026-02-25
  checkpoints_dir: /kaggle/working/outputs/2026-02-25/checkpoints
model:
  backbone: efficientnet_b0
  num_classes: 5
  pretrained: true
data:
  use_dummy: false
  image_size: 224
  train_csv: /kaggle/working/train.csv
  val_csv: /kaggle/working/val.csv
  image_dir: /kaggle/input/datasets/mohlamin/resized-eyepacs-diabetic-retinopathy-dataset/Images
  image_col: image_id
  label_col: label
  use_weighted_sampler: false
